In [2]:
import pandas as pd
import numpy as np

In [74]:
df = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/oro_evap_5yr.csv', delimiter=',',  usecols=[1])
elevation = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/oro_elevation_5yr.csv', delimiter=',',usecols=[1])
inflow = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/oro_inflow_5yr.csv', delimiter=',',usecols=[1])
outflow = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/oro_outflow_5yr.csv', delimiter=',',  usecols=[1])
ppt = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/oro_ppt_5yr.csv', delimiter=',',usecols=[1])
storage = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/oro_storage_5yr.csv', delimiter=',',  usecols=[1])

df['inflow_cfs'] = inflow
df['elevation_feet'] = elevation
df['ppt_inches'] = ppt
df['storage_taf'] = storage
df['outflow_cfs'] = outflow

df = df.rename(columns={"EVAP CFS": "evap_cfs"})

In [48]:
# df = pd.read_csv('D:/UC DAVIS/ECI 273/project/data/clean_data.csv', delimiter=',')

In [57]:
#### See which columns have the most -- values
col = df.columns
for i in range (len(col)):
    
    print (df.loc[lambda df: df[col[i]] == '--', :])


Empty DataFrame
Columns: [evap_cfs, inflow_cfs, elevation_feet, ppt_inches, storage_taf, outflow_cfs]
Index: []
Empty DataFrame
Columns: [evap_cfs, inflow_cfs, elevation_feet, ppt_inches, storage_taf, outflow_cfs]
Index: []
      evap_cfs  inflow_cfs elevation_feet ppt_inches storage_taf  outflow_cfs
605      122.0      2632.0             --       0.00  2217682.00       5955.0
773        9.0      1608.0             --       0.22  1011378.00       2459.0
774        0.0      3023.0             --       0.19  1010785.00       2725.0
775        5.0      2890.0             --       0.00  1011506.00       2522.0
776        5.0      2918.0             --       0.16  1012294.00       2516.0
871       22.0     26497.0             --       0.15  2805264.00      11194.0
1622      50.0      2362.0             --         --          --       1232.0
      evap_cfs  inflow_cfs elevation_feet ppt_inches storage_taf  outflow_cfs
88        27.0     20210.0         848.30         --  2785613.00      2577

In [75]:
df.drop(df[df['ppt_inches']=='--'].index,inplace=True)
df.drop(df[df['elevation_feet']=='--'].index,inplace=True)
df.drop(df[df['storage_taf']=='--'].index,inplace=True)

df = df.reset_index(drop=True)

In [76]:
### convert this to floats for some reason the values are strings
df['elevation_feet'] = df['elevation_feet'].astype(float)
df['storage_taf'] = df['storage_taf'].astype(float)
df['ppt_inches'] = df['ppt_inches'].astype(float)

In [77]:
df

,evap_cfs,inflow_cfs,elevation_feet,ppt_inches,storage_taf,outflow_cfs
0,17.0,2414.0,736.08,0.00,1551207.0,3555.0
1,13.0,2082.0,735.70,0.00,1547855.0,3847.0
2,39.0,1602.0,735.30,0.00,1544332.0,3339.0
3,23.0,2017.0,734.91,0.00,1540902.0,3722.0
4,23.0,1912.0,734.49,0.00,1537215.0,3703.0
...,...,...,...,...,...,...
1798,15.0,3023.0,661.99,0.05,987355.0,2000.0
1799,19.0,3518.0,662.46,1.13,990393.0,2000.0
1800,22.0,3685.0,662.99,0.00,993827.0,1900.0
1801,10.0,2594.0,663.23,0.07,995384.0,1800.0


In [42]:
# convert to feet

In [78]:
df['storage_taf'] *= 4.356*10**7
df['ppt_inches'] /= 12


In [79]:
df = df.rename(columns={"ppt_inches": "ppt_feet"})
df = df.rename(columns={"storage_taf": "storage_ft"})

## Next we categorize the data

In [80]:
#dividing cars into 3 categories by weight
min_wt = min(df['inflow_cfs'])
max_wt = max(df['inflow_cfs'])
diff = (max_wt - min_wt)/4

bin1_start = min(df['inflow_cfs'])
bin1_end = bin1_start + diff*0.10
bin2_start = max_wt - diff*0.10

def label_wt(row):
    if bin1_start <= row['inflow_cfs'] and row['inflow_cfs'] <= bin1_end:
        return 'drought'
    elif row['inflow_cfs'] <= bin2_start:
        return 'okay'
    elif bin2_start <= row['inflow_cfs'] <= max_wt:
        return 'flood'
    else:
        return 'inflow_cfs'
df['inflow_catego'] = df.apply(lambda row: label_wt(row),axis=1)

In [81]:
#dividing cars into 3 categories by weight
min_wt = min(df['outflow_cfs'])
max_wt = max(df['outflow_cfs'])
diff = (max_wt - min_wt)/4

bin1_start = min(df['outflow_cfs'])
bin1_end = bin1_start + diff*0.10
bin2_start = max_wt - diff*0.10

def label_wt(row):
    if bin1_start <= row['outflow_cfs'] and row['outflow_cfs'] <= bin1_end:
        return 'drought'
    elif row['outflow_cfs'] <= bin2_start:
        return 'okay'
    elif bin2_start <= row['outflow_cfs'] <= max_wt:
        return 'flood'
    else:
        return 'outflow_cfs'
df['outflow_catego'] = df.apply(lambda row: label_wt(row),axis=1)

In [82]:
df

,evap_cfs,inflow_cfs,elevation_feet,ppt_feet,storage_ft,outflow_cfs,inflow_catego,outflow_catego
0,17.0,2414.0,736.08,0.000000,6.757058e+13,3555.0,drought,okay
1,13.0,2082.0,735.70,0.000000,6.742456e+13,3847.0,drought,okay
2,39.0,1602.0,735.30,0.000000,6.727110e+13,3339.0,drought,okay
3,23.0,2017.0,734.91,0.000000,6.712169e+13,3722.0,drought,okay
4,23.0,1912.0,734.49,0.000000,6.696109e+13,3703.0,drought,okay
...,...,...,...,...,...,...,...,...
1798,15.0,3023.0,661.99,0.004167,4.300918e+13,2000.0,okay,drought
1799,19.0,3518.0,662.46,0.094167,4.314152e+13,2000.0,okay,drought
1800,22.0,3685.0,662.99,0.000000,4.329110e+13,1900.0,okay,drought
1801,10.0,2594.0,663.23,0.005833,4.335893e+13,1800.0,drought,drought


In [83]:
# save the clean csv file

## Convert to lognormal
for c in df.columns:
    for i in range(len(df['evap_cfs'])):
        if df[c][i] == 0:
            pass
        else:
            df[c][i] = np.log10(df[c][i])

<ipython-input-83-4ac9b4249c2e>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[c][i] = np.log10(df[c][i])
<ipython-input-83-4ac9b4249c2e>:9: RuntimeWarning: invalid value encountered in log10
  df[c][i] = np.log10(df[c][i])


TypeError: ufunc 'log10' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [90]:

df = df.rename(columns={"evap_cfs": "evaporation", "inflow_cfs": "inflow", "elevation_feet": "elevation","storage_ft": "storage","outflow_cfs": "outflow" })

df.to_csv(r'D:/UC DAVIS/ECI 273/project/data/clean_data.csv', index=False)


In [89]:
df

,evap_cfs,inflow_cfs,elevation_feet,storage_ft,outflow_cfs,inflow_catego,outflow_catego
0,1.230449,3.382737,2.866925,13.829758,3.550840,drought,okay
1,1.113943,3.318481,2.866701,13.828818,3.585122,drought,okay
2,1.591065,3.204663,2.866465,13.827829,3.523616,drought,okay
3,1.361728,3.304706,2.866234,13.826863,3.570776,drought,okay
4,1.361728,3.281488,2.865986,13.825822,3.568554,drought,okay
...,...,...,...,...,...,...,...
1798,1.176091,3.480438,2.820851,13.633561,3.301030,okay,drought
1799,1.278754,3.546296,2.821160,13.634895,3.301030,okay,drought
1800,1.342423,3.566437,2.821507,13.636399,3.278754,okay,drought
1801,1.000000,3.413970,2.821664,13.637079,3.255273,drought,drought
